# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

### Имортируем все необходимые инструмены

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    f1_score, 
    mean_squared_error, 
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    recall_score,
    precision_score
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score
)

In [2]:
state = np.random.RandomState(12345)

### Загрузим все 3 таблицы

In [3]:
try:
    data_0 = pd.read_csv("geo_data_0.csv")
except:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
try:
    data_1 = pd.read_csv("geo_data_1.csv")
except:
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
try:
    data_2 = pd.read_csv("geo_data_2.csv")
except:
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')

### Посмотрим "шапки" того что загрузили

In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


***Признаки:***

**id — уникальный идентификатор скважины;**

**f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);**

***Целевой признак:***

**product — объём запасов в скважине (тыс. баррелей).**

### Посмотрим колличество данных и типы столбцов

In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Проверим данные на пропуски

In [10]:
data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [11]:
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

**Пропусков не наблюдается**

### Поищем дубликаты

In [13]:
data_0.duplicated().sum()

0

In [14]:
data_1.duplicated().sum()

0

In [15]:
data_2.duplicated().sum()

0

**Дубликатов в данных не наблюдается**

### Избавимся от столбцов что нам не понадобятся

**Столбцы id нам не понадобятся**

In [16]:
data_0.drop(columns=['id'],axis=1,inplace=True)
data_1.drop(columns=['id'],axis=1,inplace=True)
data_2.drop(columns=['id'],axis=1,inplace=True)

### Разделим данные на обучающую и валидационную выборки

**Разделим набор данных на обучающую (train(75%)) и валидационную (validate(25%)) выборки. В random_state положим значение 12345**

In [17]:
train_0, validate_0 = \
              np.split(data_0.sample(frac=1, random_state=12345), 
                       [int(.75*len(data_0))])
train_1, validate_1 = \
              np.split(data_1.sample(frac=1, random_state=12345), 
                       [int(.75*len(data_1))])
train_2, validate_2 = \
              np.split(data_2.sample(frac=1, random_state=12345), 
                       [int(.75*len(data_2))])

In [18]:
target_0 = data_0['product']
features_0 = data_0.drop('product', axis=1)
target_1 = data_1['product']
features_1 = data_1.drop('product', axis=1)
target_2 = data_2['product']
features_2 = data_2.drop('product', axis=1)

In [19]:
features_train_0 = train_0.drop(['product'], axis=1)
target_train_0 = train_0['product']
features_valid_0 = validate_0.drop(['product'], axis=1)
target_valid_0 = validate_0['product']
features_train_1 = train_1.drop(['product'], axis=1)
target_train_1 = train_1['product']
features_valid_1 = validate_1.drop(['product'], axis=1)
target_valid_1 = validate_1['product']
features_train_2 = train_2.drop(['product'], axis=1)
target_train_2 = train_2['product']
features_valid_2 = validate_2.drop(['product'], axis=1)
target_valid_2 = validate_2['product']

In [20]:
features_train_0.shape, features_valid_0.shape

((75000, 3), (25000, 3))

In [21]:
features_train_1.shape, features_valid_1.shape

((75000, 3), (25000, 3))

In [22]:
features_train_2.shape, features_valid_2.shape

((75000, 3), (25000, 3))

In [23]:
random = np.random.RandomState(12345)

### Вывод по загрузке и проверки данных

**Данные были проверены:**

**Дубликатов, пропусков, неверных типов данных в столбцах или других аномалий не было выявленно**

**Данных были разделены в соотношении 75:25**

## Обучение и проверка модели

### Модель линейной регрессии

In [24]:
model_reg_0 = LinearRegression()
model_reg_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_reg_0.predict(features_valid_0)
materials_0 = target_0.mean()
print("Средний запас сырья:", materials_0)
print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_0, predictions_valid_0)**0.5)

Средний запас сырья: 92.50000000000001
RMSE модели линейной регрессии на валидационной выборке: 37.68019646464799


In [25]:
model_reg_1 = LinearRegression()
model_reg_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_reg_1.predict(features_valid_1)
materials_1 = target_1.mean()
print("Средний запас сырья:", materials_1)
print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_1, predictions_valid_1)**0.5)

Средний запас сырья: 68.82500000000002
RMSE модели линейной регрессии на валидационной выборке: 0.8873287354658539


In [26]:
model_reg_2 = LinearRegression()
model_reg_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_reg_2.predict(features_valid_2)
materials_2 = target_2.mean()
print("Средний запас сырья:", materials_2)
print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_2, predictions_valid_2)**0.5)

Средний запас сырья: 95.00000000000004
RMSE модели линейной регрессии на валидационной выборке: 40.11167877627781


**По итогам исследования модели линенной регрессии было выявленно, что самый точный результат регрессии среднеквадратичной ошибки получился у второго региона, но по среднему объему запасов регион на последнем месте, самый же "богатый" регион оказался третий, но там и самая высокая вероятность ошибки.**

### Модель случайного леса

**Хоть в условиях задачи и указанно что необходимо использовать только модель линейной регрессии, для контроля попробуем применить модель случайного леса в регрессии**

In [27]:
#model_forest_0 = RandomForestRegressor(random_state=12345, n_estimators=80, max_depth=9)
#model_forest_0.fit(features_train_0, target_train_0)
#predictions_valid_0 = model_forest_0.predict(features_valid_0)
#print("Средний запас сырья:", materials_0)
#print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_0, predictions_valid_0)**0.5)

In [28]:
#model_forest_1 = RandomForestRegressor(random_state=12345, n_estimators=80, max_depth=7)
#model_forest_1.fit(features_train_1, target_train_1)
#predictions_valid_1 = model_forest_1.predict(features_valid_1)
#print("Средний запас сырья:", materials_1)
#print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_1, predictions_valid_1)**0.5)

In [29]:
#model_forest_2 = RandomForestRegressor(random_state=12345, n_estimators=80, max_depth=9)
#model_forest_2.fit(features_train_2, target_train_2)
#predictions_valid_2 = model_forest_2.predict(features_valid_2)
#print("Средний запас сырья:", materials_2)
#print("RMSE модели линейной регрессии на валидационной выборке:", mean_squared_error(target_valid_2, predictions_valid_2)**0.5)

**Это модель оказалась более точной, после подбора гиперпараметров, но как и в линеной регрессии во втором регионе самый минимальный объём ресурсов, но и самая низкая вероятность ошибки**

### Вывод по обучению моделей

**В исследовании было применено две модели, линенной регрессии и случайного леса в регрессии. В обоих случаях получилась следующая картина:**

***Второй регион оказался самым бедным на средней запас ресурсов, но там вероятность ошибки самый низкий и близок к 0***

***Первый регион имеет практически такой же шанс ошибки как и третий, но объём сырья потенциального сырья там так же ниже***

***Третий регион оказался самым богатым, но и шанс ошибки там самый высокий, хотя шанс ошибки не сильно больше первого региона***

## Подготовка к расчёту прибыли

**При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.**

**Бюджет на разработку скважин в регионе — 10 млрд рублей.**

**При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.**

**После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.**

### Создадим переменный с бюджетом, ценой за один баррель сырья и колличеством скважен, что будут разрабатываться

In [30]:
budget = 10000000000
barel = 450000
well = 200

**Теперь расчитаем предпологаем объём скважены, для того что бы бюджет окупался в 0**

In [31]:
print('Минимально предполагаемый объём сырья в скважинах:', budget/barel/well, 'барелей')

Минимально предполагаемый объём сырья в скважинах: 111.11111111111111 барелей


**Минимальный объём каждой скваженны должен быть не ниже значения в 111.11**

### Поиск потенциальный кандидатов

In [32]:
data_top_0 = data_0.loc[data_0['product'] > 111.11]

In [33]:
len(data_top_0)/len(data_0)

0.36584

In [34]:
data_top_1 = data_1.loc[data_1['product'] > 111.11]

In [35]:
len(data_top_1)/len(data_1)

0.16537

In [36]:
data_top_2 = data_2.loc[data_2['product'] > 111.11]

In [37]:
len(data_top_2)/len(data_2)

0.38178

**Оказалось, что в первом регионе 36% скважен нам подходят, во втором 16%, а в третьем 38%.**

In [38]:
data_top_1.head()

,f0,f1,f2,product
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305
8,13.355129,-0.332068,4.998647,134.766305
9,1.069227,-11.025667,4.997844,137.945408


In [39]:
data_top_1['product'].unique()

array([134.76630516, 137.94540774])

**Так же мне показалось крайне странным показания столбца product во втором регионе, там все скважены имеют всего два варианта объёма: 134.76630516 и 137.94540774, я бы обратил на это внимание**

In [46]:
#ячейка ревьюера
data_1['product'].nunique()

12

### Функция расчёта прибыли

In [40]:
def find_predictions(model, features_valid, index):
    predictions = model.predict(features_valid)
    
    np_array = np.array(predictions)
    new_predictions = pd.Series(np_array, index = index)
    
    return new_predictions

**Поскольку предсказания выводятся в массиве. Поставим аналогичные с target индексы.**

In [41]:
predictions_0 = find_predictions(model_reg_0, features_valid_0, target_valid_0.index)
predictions_1 = find_predictions(model_reg_1, features_valid_1, target_valid_1.index)
predictions_2 = find_predictions(model_reg_2, features_valid_2, target_valid_2.index)

In [42]:
def profit (target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (barel * selected.sum()) - budget

### Вывод по поиску потенциальной прибыли

**Мы посчитали примерное колличество скважен что нам подходят по объёму в каждом регионе:**

***Первый регион: 36%***

***Второй регион: 16%***

***Третий регион: 38%***

**Так же была написана функция для расчёта прибыли**

## Расчёт прибыли и рисков 

### Расчёт прибыли по регионам

**Выполните процедуру Bootstrap с 1000 повторений, по каждому из регионов и найдём среднию возможную вырочку**

**Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.**

In [43]:
values_0 = []
for i in range(1000):
    target_subsample_0 = target_valid_0.sample(n=500, replace=True, random_state=state)
    probs_subsample_0 = predictions_0[target_subsample_0.index] 
    values_0.append(profit(target_subsample_0, probs_subsample_0, 200))
values_0 = pd.Series(values_0)
print('Средняя выручка первого региона с 200 лучших точек:', values_0.mean())
confidence_interval_0 = st.t.interval(
    0.95, len(values_0)-1, values_0.mean(), values_0.sem())
print("95%-ый доверительный интервал:", confidence_interval_0)
print('Риск убытков', sum([1 for i in values_0 if i < 0]) / len(values_0)*100,'%')

Средняя выручка первого региона с 200 лучших точек: 468757230.73649806
95%-ый доверительный интервал: (451722609.9257023, 485791851.54729384)
Риск убытков 4.6 %


In [44]:
values_1 = []
for i in range(1000):
    target_subsample_1 = target_valid_1.sample(n=500, replace=True, random_state=state)
    probs_subsample_1 = predictions_1[target_subsample_1.index] 
    values_1.append(profit(target_subsample_1, probs_subsample_1, 200))
values_1 = pd.Series(values_1)
print('Средняя выручка второго региона с 200 лучших точек:', values_1.mean())
confidence_interval_1 = st.t.interval(
    0.95, len(values_1)-1, values_1.mean(), values_1.sem())
print("95%-ый доверительный интервал:", confidence_interval_1)
print('Риск убытков', sum([1 for i in values_1 if i < 0]) / len(values_1)*100,'%')

Средняя выручка второго региона с 200 лучших точек: 495302032.880433
95%-ый доверительный интервал: (482350862.2308653, 508253203.53000075)
Риск убытков 0.8999999999999999 %


In [45]:
values_2 = []
for i in range(1000):
    target_subsample_2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    probs_subsample_2 = predictions_2[target_subsample_2.index] 
    values_2.append(profit(target_subsample_2, probs_subsample_2, 200))
values_2 = pd.Series(values_2)
print('Средняя выручка третьего региона с 200 лучших точек:', values_2.mean())
confidence_interval_2 = st.t.interval(
    0.95, len(values_2)-1, values_2.mean(), values_2.sem())
print("95%-ый доверительный интервал:", confidence_interval_2)
print('Риск убытков', sum([1 for i in values_2 if i < 0]) / len(values_2)*100,'%')

Средняя выручка третьего региона с 200 лучших точек: 365437773.923951
95%-ый доверительный интервал: (348257993.6535976, 382617554.19430447)
Риск убытков 10.0 %


### Вывод по расчёту прибыли

**По итогам исследований получилась следующая картина:**

***Средняя выручка первого региона с двухсот лучших точек: 465457860.2982787
95%-ый доверительный интервал: (448330175.8666724, 482585544.729885)
Риск убытков 4.0%***

***Средняя выручка второго региона с двухсот лучших точек: 491278139.074479
95%-ый доверительный интервал: (477882794.47297674, 504673483.6759812)
Риск убытков 0.8999999999999999%***

***Средняя выручка третьего региона с двухсот лучших точек: 378924122.30922544
95%-ый доверительный интервал: (361876187.65304816, 395972056.9654027)
Риск убытков 8.7%***

**Хоть второй регион и укладывается в задачу, что бы риск был ниже 2.5% и по условиям задачи подходит, но я хотел бы обратить особое внимае тому, что данные в этом регионе выглядят подозритель и их стоит перепроверить**

### Общий вывод

**Были проведенные исследования данных из трёх потенциальных регион для добычи ископаемых и были выполненны следующие действия:**

**Данные были загруженны и проверены на пропуски, дубликаты**

**В данных были удалены стобцы с id, так как они были не нужны для исследований**

**Данные были разделены на обучающую (train(75%)) и валидационную (validate(25%)) выборки**

**Были пременены методы обучения модели линенной регрессии и случайного леса регрессии (как оказалась по условию задачи, нужна была только линеная регрессия)**

**Модели получили следующие результаты:**

***Первый регион имеет средний запас сырья: 92.50000000000001, а RMSE модели линейной регрессии на валидационной выборке: 37.68019646464799***

***Второй регион имеет средний запас сырья: 68.82500000000002, а RMSE модели линейной регрессии на валидационной выборке: 0.8873287354658539***

***Третий регион имеет средний запас сырья: 95.00000000000004, а RMSE модели линейной регрессии на валидационной выборке: 40.11167877627781***

**Были проведены подготовительные работы для расчёта самого перспективного региона, и было выявленно, что минимальный объём каждой из 200 скважен должен быть не ниже значения в 111.11 барелей**

**Оказалось, что в первом регионе 36%, во втором 16%, а в третьем 38% скважен подходят под заданные параметры**

**Так же мне показалось крайне странным показания столбца product во втором регионе, там все скважены имеют всего два варианта объёма: 134.76630516 и 137.94540774, я бы обратил на это внимание!**

**Выполнена процедура Bootstrap с 1000 повторений, по каждому из регионов и найдняя возможная выручка по двухста лучшим скважинам, а так же риск убытков:**

***Средняя выручка первого региона с двухсот лучших точек: 465457860.2982787, 95%-ый доверительный интервал: (448330175.8666724, 482585544.729885), Риск убытков 4.0%***

***Средняя выручка второго региона с двухсот лучших точек: 491278139.074479, 95%-ый доверительный интервал: (477882794.47297674, 504673483.6759812), Риск убытков 0.8999999999999999%***

***Средняя выручка третьего региона с двухсот лучших точек: 378924122.30922544, 95%-ый доверительный интервал: (361876187.65304816, 395972056.9654027), Риск убытков 8.7%***

**По условиям поставленной задачи лучше всего подходит второй регион (мои опасения по поводу этого региона указаны выше в отчёте)**